In [153]:
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pandas as pd
import time

In [ ]:
# def init_browser():
    executable_path = {'executable_path': 'chromedriver.exe'}
    return Browser('chrome', **executable_path, headless=False)

def scrape():
    browser = init_browser()
    mars_scraped_data_list = []

    #MARS NEWS
    news_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
    response = requests.get(news_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraph_results = soup.find_all('div', class_="rollover_description_inner") 
    title_results = soup.find_all('div', class_="content_title")
    news_title = title_results[0].text.strip()
    news_paragraph = paragraph_results[0].text.strip()
    

    #JPL Mars Space Images - Featured Image
    image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(image_url)
    time.sleep(3)
    html = browser.html
    soup = BeautifulSoup(html,'html.parser')
    result = soup.find('div', class_='img')
    featured_image_url = result.img['src']


    #Mars Weather
    weather_url = 'https://twitter.com/marswxreport?lang=en'
    browser.visit(weather_url)
    time.sleep(3)
    tw_html = browser.html
    soup = BeautifulSoup(tw_html, 'html.parser')
    tw_result = soup.find_all('span', class_='css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0')
    mars_weather = tw_result[27].text
    

    #Mars Facts
    facts_url = 'https://space-facts.com/mars/'
    tables = pd.read_html(facts_url)
    mars_facts_df = tables[2]
    mars_facts_df = mars_facts_df.rename(columns = {0:'Description', 1:'Fact'})
    #mars_facts_df.set_index('Description', inplace=True)  
    mars_facts_html_table = mars_facts_df.to_html(index=False) 
    mars_facts_html_table.replace('\n', '')
    print(mars_facts_html_table)
    #mars_facts_df.to_html('mars_facts_table.html')
    
    #Mars Hemispheres
    hemi_list = ['Cerberus','Schiaparelli','Syrtis','Valles']
    hemi_img_url = [] 
    
    for x in hemi_list:
        hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
        browser.visit(hemi_url)
        time.sleep(3)
        browser.links.find_by_partial_text(x).click()
        browser.find_link_by_text('Open').first.click()
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        img_url = soup.find('img', class_='wide-image')['src']
        title = soup.find('h2', class_='title').text
        hemi_img_url.append({'title':title, 'img_url':img_url})
        
    print(hemi_img_url)
    
    record = {
                'title':  news_title,
                'paragraph': news_paragraph,
                'image': featured_image_url,
                'weather': mars_weather,
                'hemi_image':hemi_img_url,
                'fact_table':mars_facts_html_table
            }
    print(mars_scraped_data_list)
    print('--- testing ---')
    print(record)
    mars_scraped_data_list.append(record)
    

    return mars_scraped_data_list